Tutorial - MCP Server
======================

ROSS provides a [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) server that exposes core rotordynamics analysis tools for use with AI assistants such as Claude, GitHub Copilot, or any MCP-compatible client.

This allows you to interact with ROSS using natural language prompts, making it easy to run analyses without writing Python code directly.

## Installation

Install ROSS with the MCP optional dependency:

```bash
pip install ross-rotordynamics[mcp]
```

## Configuration

To use the MCP server with an AI assistant, create a `.mcp.json` file in your project root:

```json
{
  "mcpServers": {
    "ross": {
      "command": "ross-mcp"
    }
  }
}
```

Alternatively, you can run the server directly:

```bash
python -m ross.mcp
```

## Available Tools

The MCP server exposes the following tools:

| Tool | Description |
| --- | --- |
| `create_example_rotor` | Load a pre-built example rotor (default, compressor, 6dof, damped) |
| `describe_rotor` | Get a detailed description of a stored rotor (elements, masses, DOFs) |
| `run_modal_analysis` | Eigenvalue analysis at a given speed |
| `run_campbell_diagram` | Damped natural frequencies over a speed range |
| `run_unbalance_response` | Forced response to unbalance excitation |

## Example

The following example demonstrates how a natural language prompt is handled by the MCP server.

**Prompt:**

> Give the natural frequencies (rpm) for the compressor running at 10000 rpm

Behind the scenes, the AI assistant calls two MCP tools. First, it creates the compressor rotor:

In [1]:
# Tool call: create_example_rotor(name="compressor", variant="compressor")
import ross as rs

rotor = rs.compressor_example()
print(
    f"Rotor 'compressor': {len(rotor.shaft_elements)} shaft elements, "
    f"{len(rotor.disk_elements)} disks, "
    f"{len(rotor.bearing_elements)} bearings, "
    f"{len(rotor.nodes)} nodes, "
    f"{rotor.ndof} DOFs, "
    f"total length = {rotor.L:.4f} m"
)

Rotor 'compressor': 91 shaft elements, 7 disks, 14 bearings, 56 nodes, 336 DOFs, total length = 1.6533 m


Then, it runs the modal analysis at 10000 RPM (1047.2 rad/s) and converts the results to RPM:

In [2]:
# Tool call: run_modal_analysis(name="compressor", speed=1047.2, num_modes=12)
import numpy as np

speed = 10000 * 2 * np.pi / 60  # 10000 RPM to rad/s
modal = rotor.run_modal(speed=speed, num_modes=12)

# Convert natural frequencies from rad/s to RPM
wn_rpm = modal.wn * 60 / (2 * np.pi)

print("Natural frequencies for the compressor at 10000 RPM:\n")
print(
    f"| {'Mode':>4s} | {'Natural Freq (RPM)':>20s} | {'Damping Ratio':>15s} | {'Log Dec':>9s} |"
)
print(
    f"|{'------|':->6s}{'----------------------|':->22s}{'-----------------':->17s}|{'-----------':->11s}|"
)
for i in range(len(modal.wn)):
    print(
        f"| {i + 1:>4d} | {wn_rpm[i]:>20.1f} | {modal.damping_ratio[i]:>15.3f} | {modal.log_dec[i]:>9.2f} |"
    )

Natural frequencies for the compressor at 10000 RPM:

| Mode |   Natural Freq (RPM) |   Damping Ratio |   Log Dec |
|------|----------------------|-----------------|-----------|
|    1 |              10055.1 |           0.278 |      1.82 |
|    2 |              10016.5 |           0.102 |      0.64 |
|    3 |              19035.2 |           0.548 |      4.11 |
|    4 |              19332.1 |           0.541 |      4.04 |
|    5 |              18197.6 |           0.387 |      2.64 |
|    6 |              18695.3 |           0.412 |      2.84 |


**AI assistant response:**

Here are the natural frequencies for the compressor rotor at 10,000 RPM:

| Mode | Natural Freq (RPM) | Damping Ratio | Log Dec |
| --- | --- | --- | --- |
| 1 | 10,055 | 0.278 | 1.82 |
| 2 | 10,016 | 0.102 | 0.64 |
| 3 | 19,035 | 0.548 | 4.11 |
| 4 | 19,332 | 0.541 | 4.04 |
| 5 | 18,198 | 0.387 | 2.64 |
| 6 | 18,695 | 0.412 | 2.84 |

At 10,000 RPM the rotor is operating very close to the first two natural frequencies (~10,055 and ~10,016 RPM), which means it is near a critical speed. All modes show positive log decrements, indicating stable operation.